<a href="https://colab.research.google.com/github/mulcahrj/DATA6545_Final/blob/main/Logistical_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = 'https://raw.githubusercontent.com/mulcahrj/DATA6545_Final/main/selected_features.csv'

features = pd.read_csv(path)

In [ ]:
features.head()

,emp.var.rate,cons.conf.idx,euribor3m,total_assets,rep_avg_rating,customer_length,poutcome_success,customer_ratings,housing_yes,cons.price.idx,y
0,1.000000,0.408442,0.967665,0.553302,0.740661,0.787138,0,5.0,1,0.605052,0
1,0.148320,0.235723,0.061928,0.817121,0.275429,0.357026,0,3.0,1,0.534818,0
2,1.000000,0.692481,0.968428,0.191640,0.959432,0.966685,0,5.0,1,0.414483,0
3,0.849418,0.680736,0.929140,0.399434,0.285757,0.369019,0,3.0,1,0.637496,0
4,1.000000,0.408442,0.967665,0.291517,0.244337,0.325443,0,3.0,0,0.605052,0


In [ ]:
X = features.iloc[:,:-1].values
y = features.iloc[:,-1].values

In [ ]:
!pip install imblearn==0.0

In [ ]:
# import required packages for splitting data
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split



# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `Decision Tree` model
from sklearn.tree import DecisionTreeClassifier

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

# import 'XG Boost' model
import xgboost as xgb

# import 'random forest' model
from sklearn.ensemble import RandomForestClassifier

#import 'SVM' model
from sklearn import svm

#import 'Naive Bayes' model
from sklearn.naive_bayes import GaussianNB

# balance the data
from imblearn.over_sampling import SMOTE

### Evaluation Function

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
#function inputs X:features.Y: Target, classifer: sklearn classifier, K:folds in cross validation, scoring: evaluation metric(f1)
def my_eval(X, y, classifer = clf, k=10, scoring = 'f1'):
    scores = []
    for i in range(5):
        #### generate random numbers to shuffle the data for training and test
        np.random.seed(2021)
        random_int = np.random.randint(0,3000)
        #### create cross validation folds
        kfold = model_selection.KFold(n_splits=k, random_state=random_int, shuffle=True)
        #### record the score 
        score = model_selection.cross_val_score(clf, X=X, y=y, cv=kfold, scoring=scoring)
        scores.append(score)
    scores = np.array(scores)
    #### we need to calculate the bias (average score) and viariance (std)
    bias, variance = round(scores.mean(),4), round(scores.std(),4)
    return(bias, variance)

###Smote

In [ ]:
# Resampling and Balancing the data
sm = SMOTE(random_state = 2021) 
X_res, Y_res = sm.fit_resample(X, y) 

##Train Test Split

In [ ]:
#Creating a Train,Test data set 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=2019) 

## Decision Tree

In [ ]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [ ]:
#average f1_score from 10-fold CV 
clf = LogisticRegression(max_iter=2000)
my_eval(X_res, Y_res, clf, 10)

(0.8993, 0.0042)

In [ ]:
#average ROC_AUC from 10-fold CV
my_eval(X_res, Y_res, clf, 10, 'roc_auc')

(0.8986, 0.0039)